In [16]:
# http://qwone.com/~jason/20Newsgroups/ 下载 20-news 数据
# 20-news 数据集包含 20 个不同主题
# 20-news 数据集的目录结构如下：
# 20news-18828
# ├── alt.atheism
# ├── comp.graphics
# ├── comp.os.ms-windows.misc
# ├── comp.sys.ibm.pc.hardware
# ├── comp.sys.mac.hardware
# ├── comp.windows.x
# ├── misc.forsale
# ├── rec.autos
# ├── rec.motorcycles
# ├── rec.sport.baseball
# ├── rec.sport.hockey
# ├── sci.crypt
# ├── sci.electronics
# ├── sci.med
# ├── sci.space
# ├── soc.religion.christian
# ├── talk.politics.guns
# ├── talk.politics.mideast
# ├── talk.politics.misc
# └── talk.religion.misc


from pyspark.sql import SparkSession
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

spark = SparkSession.builder.appName("lecture3").getOrCreate()

sc = spark.sparkContext

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# 试用前需要下载 nltk 的 stop_words
# import nltk
# nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mikkel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
def remove_punctuation(text):
    # 删除标点符号
    import string
    return text.translate(str.maketrans('', '', string.punctuation))

In [19]:
remove_punctuation("Hi, I'm peter!")

'Hi Im peter'

In [20]:
rdd = sc.wholeTextFiles('20news-18828/comp.graphics/*')

In [21]:
# map step in mapreduce
# 转换成小写
rdd = rdd.map(lambda x: x[1].lower())
rdd.take(1)
# 分词
rdd = rdd.flatMap(lambda x: x.split())
rdd.take(1)
# 去除停用词
rdd = rdd.filter(lambda x: x not in stop_words)
# 去除标点符号
rdd = rdd.map(remove_punctuation)
# 去掉停用词和空字符串
rdd = rdd.filter(lambda x: x not in stop_words)
rdd = rdd.filter(lambda x: x != '')
# 每个词出现就计1
rdd = rdd.map(lambda x: (x, 1))
rdd.take(1)

Py4JJavaError: An error occurred while calling o178.partitions.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input Pattern file:/Users/mikkel/3_Projects/Learning-Hub/Learning-Hub/20news-18828/comp.graphics/* matches 0 files
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:340)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:279)
	at org.apache.spark.input.WholeTextFileInputFormat.setMinPartitions(WholeTextFileInputFormat.scala:52)
	at org.apache.spark.rdd.WholeTextFileRDD.getPartitions(WholeTextFileRDD.scala:54)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org

23/10/17 02:34:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1025635 ms exceeds timeout 120000 ms
23/10/17 02:34:28 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/17 02:34:30 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [ ]:
# reduce step in mapreduce
# 按照 key 进行聚合
rdd = rdd.reduceByKey(lambda x, y: x + y)
# 按照词频排序
rdd = rdd.sortBy(lambda x: x[1], ascending=False)
rdd.take(200)

Py4JJavaError: An error occurred while calling o26.partitions.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input Pattern file:/Users/mikkel/3_Projects/Learning-Hub/Learning-Hub/20news-18828/comp.graphics/* matches 0 files
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:340)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:279)
	at org.apache.spark.input.WholeTextFileInputFormat.setMinPartitions(WholeTextFileInputFormat.scala:52)
	at org.apache.spark.rdd.WholeTextFileRDD.getPartitions(WholeTextFileRDD.scala:54)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.

In [ ]:
# RDD 的 Join 操作

rdd1 = sc.parallelize([(1001, 'Mike'), (1002, 'Peter'), (1003, 'John'), (1004, 'Mary')])
rdd2 = sc.parallelize([(1001, 'Sales'), (1002, 'IT'), (1005, 'Engineering')])

rdd1.join(rdd2).collect()

rdd1.leftOuterJoin(rdd2).collect()

rdd1.rightOuterJoin(rdd2).collect()

[(1001, ('Mike', 'Sales')), (1002, ('Peter', 'IT'))]

[(1001, ('Mike', 'Sales')),
 (1002, ('Peter', 'IT')),
 (1003, ('John', None)),
 (1004, ('Mary', None))]

[(1001, ('Mike', 'Sales')),
 (1002, ('Peter', 'IT')),
 (1005, (None, 'Engineering'))]